# Missionars and Cannibals - a lab activity on search strategies
---

## Prerequisites

- Python 3 (correctly installed)

- the AIMA libraries for search; two alternatives:
    - either (git-)clone the AIMA repository: <https://github.com/aimacode/aima-python>
    - or download from the course site the `search.py` file

- module `numpy`

- file `reporting.py`: contains some useful functions for inspecting what's going on

- file `utils.py`: it is used by some search implementations, simply place it in the same folder.

---


## The AIMA search library

The library provided with the textbook contains an implementation of the most notable search strategies.

To use them, two main steps:
1. Decide a representation of a state. A state can be represented with any object you like. However, in order to correctly use the Graph-search algorithms, the `__eq__()` method should be overrided correctly. Suggestion: in the first exercise, opt to represent the state using a tuple.

2. Define a new class, that will inherit form the class `Problem`, and that will contain the methods and the attributes needed to apply the search strategies. In particular, our new subclass of `Problem` will have to implement/override:
    - constructor `def __init__(self, initial, goal=None)`: """The constructor specifies the initial state, and possibly a goal state, if there is a unique goal. Your subclass's constructor can add other arguments."""
    
    - method `def actions(self, state):`
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        
    - method `def result(self, state, action):`
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""

    - method `def goal_test(self, state):`
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        
    - method `def path_cost(self, c, state1, action, state2):`
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1.
    - method `def h(self, node):`
		"""Returns the heuristic applied to the Node node"""

## The Missionars and Cannibals Problem

3 missionaries and 3 cannibals are on the same shore of a river, and want to cross such river. There is a single boat (on the same initial shore), able to bring around two persons at a time.
Should it happen that in ANY shore there are more cannibals than missionaries, the cannibals will eat the missionaries (failure states).

- Which state representation?
> Suggestion: a tuple of three numbers representing the number of missionaries, the number of cannibals, and the presence of the boat on the starting shore. 
> The initial state would be: `(3,3,1)`

- Which actions?
> The boat cross the river with passengers:
> - 1 missionary, 1 cannibal, 
> - 2 missionaries, 
> - 2 cannibals,  
> - 1 missionary 
> - 1 cannibal.

- Which is the goal?
> If the state is a tuple, the goal would be state `(0,0,0)`

- Which path cost?
> The number of river crossings might be a sensible choice.


In [4]:
from search import *
from reporting import *


class MC(Problem):
    """How to represent the state?
    Choice: state as a tuple, indicating
    the number of missionars, the number of cannibals, and the presence of the boat on the starting river
    Hence, the initial state is (3, 3, 1)
    The goal state will be (0, 0, 0)"""

    def __init__(self, initial, goal=None):
        """Constructor"""
        super().__init__(initial, goal)

    def isValid(self, state):
        '''Utility function introduced to check if a state is a valid one'''
        m, c, _b = state
        if (m>0 and c>m) or ((3-m)>0 and (3-c)>(3-m)):
            return False
        else:
            return True

    def actions(self, state):
        """The actions executable in this state."""
        if not self.isValid(state):
            return []

        m, c, b = state
        result = []
        smart = True
        if not smart:
            if m > 0 and c > 0 and b:
                result.append('MC->')
            if m > 1 and b:
                result.append('MM->')
            if c > 1 and b:
                result.append('CC->')
            if m > 0 and b:
                result.append('M->')
            if c > 0 and b:
                result.append('C->')
            if (3-m) > 0 and (3-c) > 0 and not b:
                result.append('<-MC')
            if (3-m) > 1 and not b:
                result.append('<-MM')
            if (3-c) > 1 and not b:
                result.append('<-CC')
            if (3-m) > 0 and not b:
                result.append('<-M')
            if (3-c) > 0 and not b:
                result.append('<-C')
        else: # the smarter solution... but more checks are done!!!
            if m > 0 and c > 0 and b==1 and self.isValid((m-1,c-1,0)):
                result.append('MC->')
            if m > 1 and b==1 and self.isValid((m-2,c,0)):
                result.append('MM->')
            if c > 1 and b==1 and self.isValid((m,c-2,0)):
                result.append('CC->')
            if m > 0 and b==1 and self.isValid((m-1,c,0)):
                result.append('M->')
            if c > 0 and b==1 and self.isValid((m,c-1, 0)):
                result.append('C->')
            if (3-m) > 0 and (3-c) > 0 and b==0 and self.isValid((m+1,c+1,1)):
                result.append('<-MC')
            if (3-m) > 1 and b==0 and self.isValid((m+2,c,1)):
                result.append('<-MM')
            if (3-c) > 1 and b==0 and self.isValid((m,c+2,1)):
                result.append('<-CC')
            if (3-m) > 0 and b==0 and self.isValid((m+1,c,1)):
                result.append('<-M')
            if (3-c) > 0 and b==0 and self.isValid((m,c+1,1)):
                result.append('<-C')
        return result

    def result(self, state, action):
        """The state that results from executing this action in this state."""
        m, c, b = state
        if action == 'MC->':
            return (m - 1, c - 1, 0)
        elif action == 'MM->':
            return (m-2, c, 0)
        elif action == 'CC->':
            return (m, c-2, 0)
        elif action == 'M->':
            return (m-1, c, 0)
        elif action == 'C->':
            return (m, c-1, 0)
        elif action == '<-MC':
            return (m+1, c+1, 1)
        elif action == '<-MM':
            return (m+2, c, 1)
        elif action == '<-CC':
            return (m, c+2, 1)
        elif action == '<-M':
            return (m+1, c, 1)
        elif action == '<-C':
            return (m, c+1, 1)
        else:
            print("ERRORE!!!")
            return None


    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def h(self, node):
        m, c,  b = node.state
        return m + c - b





mc1 = MC((3, 3, 1), (0, 0, 0))
soln = breadth_first_tree_search(mc1)
# print("Done!!!")
path = path_actions(soln)
print(path)
path = path_states(soln)
print(path)

report([
    breadth_first_tree_search,
    breadth_first_graph_search,
    # depth_first_tree_search,
    depth_first_graph_search,
    astar_search
    ],
    [mc1])

['MC->', '<-M', 'CC->', '<-C', 'MM->', '<-MC', 'MM->', '<-C', 'CC->', '<-M', 'MC->']
[(3, 3, 1), (2, 2, 0), (3, 2, 1), (3, 0, 0), (3, 1, 1), (1, 1, 0), (2, 2, 1), (0, 2, 0), (0, 3, 1), (0, 1, 0), (1, 1, 1), (0, 0, 0)]
breadth_first_tree_search:
   24,228 inserted nodes |   10,419 tested goals |   11 cost |  10,429 expanded_nodes | MC
   24,228 inserted nodes |   10,419 tested goals |   11 cost |  10,429 expanded_nodes | TOTAL

breadth_first_graph_search:
       28 inserted nodes |       15 tested goals |   11 cost |      24 expanded_nodes | MC
       28 inserted nodes |       15 tested goals |   11 cost |      24 expanded_nodes | TOTAL

depth_first_graph_search:
       26 inserted nodes |       13 tested goals |   11 cost |      23 expanded_nodes | MC
       26 inserted nodes |       13 tested goals |   11 cost |      23 expanded_nodes | TOTAL

astar_search:
       28 inserted nodes |       14 tested goals |   11 cost |      24 expanded_nodes | MC
       28 inserted nodes |       14 te